In [1]:
!pip install langchain langgraph cassio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 8.3 MB/s eta 0:00:00


In [2]:
import cassio

astra_db_token="MY_ASTRA_KEY"
astra_db_id="MY_DB_ID"
cassio.init(token=astra_db_token, database_id=astra_db_id)


In [3]:
!pip install -U tiktoken langchain-groq langchainhub langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0


In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader

urls = [
    "https://docs.langchain.com/oss/python/deepagents/overview?_gl=1*1s1qggf*_gcl_au*NDUzOTA2NDk4LjE3NjI1MjUxMTE.*_ga*OTgxMjQ2NzA1LjE3NjI1MjUxMTI.*_ga_47WX3HKKY2*czE3NjUwMTI4NDMkbzkkZzAkdDE3NjUwMTI4NDMkajYwJGwwJGgw#deep-agents-overview",
    "https://www.langchain.com/langgraph"
]

docs=[WebBaseLoader(url).load() for url in urls]
doc_list=[item for sublist in docs for item in sublist]
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(doc_list)

In [13]:
from google.colab import userdata
hf_key=userdata.get('HF_KEY')
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model="all-MiniLM-L6-v2")

In [18]:
from langchain_classic.vectorstores.cassandra import Cassandra
astra_vector_store = Cassandra(
    embedding=embeddings,
    session=None,
    table_name="qa_mini_demo",
    keyspace=None
)

In [20]:
from langchain_classic.indexes.vectorstore import VectorStoreIndexWrapper
astra_vector_store.add_documents(texts)
print("Inserted %i headlines" %len(texts))
astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 7 headlines


In [34]:
from IPython.display import Markdown
retriever = astra_vector_store.as_retriever()
response = retriever.invoke("what is langgraph?")
display(Markdown(response[0].page_content))

LangGraph FAQsHow is LangGraph different from other agent frameworks?

Other agentic frameworks can work for simple, generic tasks but fall short for complex tasks bespoke to a company’s needs. LangGraph provides a more expressive framework to handle companies’ unique tasks without restricting users to a single black-box cognitive architecture.Does LangGraph impact the performance of my app?

LangGraph will not add any overhead to your code and is specifically designed with streaming workflows in mind.Is LangGraph open source? Is it free?

Yes. LangGraph is an MIT-licensed open-source library and is free to use.What are my options if I want to deploy my agent?

You can deploy your LangGraph application using LangSmith Deployment, available in the LangSmith Plus and Enterprise plans. which has the following agent deployment options: Cloud: Fully managed and hosted as part of LangSmith (our unified observability & evals platform). Deploy quickly, with automatic updates and zero maintenance. ‍Hybrid (SaaS control plane, self-hosted data plane). No data leaves your VPC. Provisioning and scaling is managed as a service.‍Fully Self-Hosted: Deploy LangGraph entirely on your own infrastructure.‍Ready to start shipping  reliable agents faster?Get started with tools from the LangChain product suite for every step of the agent development lifecycle.Talk to salesSign UpProductsLangChainLangGraphLangSmith ObservabilityLangSmith EvaluationLangSmith DeploymentResourcesGuidesBlogCustomer StoriesLangChain AcademyCommunityEventsChangelogDocsSupportCompanyAboutCareersXLinkedInYouTubeMarketing AssetsSecuritySign up for our newsletter to stay up to dateThank you! Your submission has been received!Oops! Something went wrong while submitting the form.All systems operationalPrivacy PolicyTerms of Service